# ManiSkill - From Raw Data to RGBD Visualization

**Prerequisites:** Run `setup_maniskill.sh` first!

This notebook will:
1. ✅ Check installation
2. 📥 Download demo dataset
3. 🔍 Explore raw trajectory structure
4. 🔄 Replay trajectory with RGBD (with PROPER state initialization)
5. 🖼️ Visualize RGB and Depth images
6. 🎬 Watch the video
7. 📦 Replay 50 trajectories and view them

---
## 1. Check Installation

In [ ]:
# Check if everything is installed
print("=" * 50)
print("CHECKING INSTALLATION")
print("=" * 50)

# Check imports
try:
    import gymnasium as gym
    print("✓ gymnasium")
except ImportError as e:
    print(f"✗ gymnasium: {e}")

try:
    import mani_skill
    import mani_skill.envs
    print("✓ mani_skill")
except ImportError as e:
    print(f"✗ mani_skill: {e}")

try:
    import h5py
    print("✓ h5py")
except ImportError as e:
    print(f"✗ h5py: {e}")

try:
    import torch
    print(f"✓ torch (CUDA: {torch.cuda.is_available()})")
except ImportError as e:
    print(f"✗ torch: {e}")

try:
    import matplotlib.pyplot as plt
    print("✓ matplotlib")
except ImportError as e:
    print(f"✗ matplotlib: {e}")

try:
    import imageio
    print("✓ imageio")
except ImportError as e:
    print(f"✗ imageio: {e}")

try:
    from tqdm import tqdm
    print("✓ tqdm")
except ImportError as e:
    print(f"✗ tqdm: {e}")

import numpy as np
import os

print("\n" + "=" * 50)
print("All imports successful!")
print("=" * 50)

In [ ]:
# Quick environment test
print("Testing ManiSkill environment...")

env = gym.make('PickCube-v1', obs_mode='rgbd', render_mode='rgb_array', num_envs=1)
obs, _ = env.reset()

print(f"\n✓ Environment created!")
print(f"  Observation keys: {list(obs.keys())}")

if 'sensor_data' in obs:
    print(f"  Cameras available: {list(obs['sensor_data'].keys())}")
    
env.close()
print("\n✓ RGBD rendering is working!")

---
## 2. Download Demo Dataset

In [ ]:
# Download PickCube-v1 demonstrations
!python -m mani_skill.utils.download_demo "PickCube-v1"

In [ ]:
# Check what was downloaded
import os

demo_path = os.path.expanduser("~/.maniskill/demos/PickCube-v1")

print(f"Demo directory: {demo_path}")
print("\nContents:")

for root, dirs, files in os.walk(demo_path):
    level = root.replace(demo_path, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = '  ' * (level + 1)
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        print(f"{subindent}{file} ({size:.2f} MB)")

---
## 3. Explore Raw Trajectory Structure

Let's look inside the trajectory file to understand what data we have.

In [ ]:
import h5py
import os

# Path to trajectory file
traj_path = os.path.expanduser("~/.maniskill/demos/PickCube-v1/motionplanning/trajectory.h5")

print("=" * 60)
print("RAW TRAJECTORY STRUCTURE")
print("=" * 60)

def explore_h5(group, indent=0):
    """Recursively explore HDF5 structure"""
    for key in group.keys():
        item = group[key]
        prefix = "  " * indent
        
        if isinstance(item, h5py.Dataset):
            print(f"{prefix}📊 {key}: shape={item.shape}, dtype={item.dtype}")
        elif isinstance(item, h5py.Group):
            print(f"{prefix}📁 {key}/")
            explore_h5(item, indent + 1)

with h5py.File(traj_path, 'r') as f:
    # How many trajectories?
    traj_keys = [k for k in f.keys() if k.startswith('traj_')]
    print(f"\nTotal trajectories: {len(traj_keys)}")
    print(f"Trajectory names: {traj_keys[:5]} ... {traj_keys[-2:]}")
    
    # Look at first trajectory
    print(f"\n--- Structure of traj_0 ---\n")
    explore_h5(f['traj_0'])

In [ ]:
# Check if RGB/Depth images exist in raw data
with h5py.File(traj_path, 'r') as f:
    traj = f['traj_0']
    
    print("=" * 60)
    print("CHECKING FOR VISUAL DATA")
    print("=" * 60)
    
    if 'obs' in traj:
        obs_keys = list(traj['obs'].keys()) if isinstance(traj['obs'], h5py.Group) else []
        print(f"\nObservation keys: {obs_keys}")
        
        if 'sensor_data' in traj['obs']:
            print("\n✓ sensor_data EXISTS - Raw data has images!")
            print("  Cameras:", list(traj['obs/sensor_data'].keys()))
        else:
            print("\n✗ No sensor_data found")
            print("  → We need to REPLAY trajectories to get RGB/Depth images!")
    else:
        print("\n✗ No 'obs' group found")
        print("  → We need to REPLAY trajectories")

In [ ]:
# Look at one trajectory's actions and env_states
with h5py.File(traj_path, 'r') as f:
    actions = f['traj_0/actions'][:]
    success = f['traj_0/success'][:]
    
    print("=" * 60)
    print("TRAJECTORY DETAILS (traj_0)")
    print("=" * 60)
    print(f"\nNumber of timesteps: {len(actions)}")
    print(f"Action dimension: {actions.shape[1]}")
    print(f"Task successful: {success[-1]}")
    
    print(f"\nFirst action: {actions[0]}")
    print(f"Last action:  {actions[-1]}")
    
    # Check env_states - THIS IS KEY for proper replay!
    print("\n--- env_states (for proper initialization) ---")
    if 'env_states' in f['traj_0']:
        env_states = f['traj_0/env_states']
        for key in env_states.keys():
            print(f"  {key}/")
            if isinstance(env_states[key], h5py.Group):
                for subkey in env_states[key].keys():
                    shape = env_states[key][subkey].shape
                    print(f"    {subkey}: {shape}")

---
## 4. Replay Trajectory with RGBD (PROPER State Initialization)

**IMPORTANT:** We must set the environment to the EXACT initial state from the trajectory!
Otherwise the robot won't be in the right position to pick up the cube.

In [ ]:
import gymnasium as gym
import mani_skill.envs
import h5py
import numpy as np
import os
from tqdm import tqdm
import imageio
import json

# Paths
traj_path = os.path.expanduser("~/.maniskill/demos/PickCube-v1/motionplanning/trajectory.h5")
json_path = os.path.expanduser("~/.maniskill/demos/PickCube-v1/motionplanning/trajectory.json")
output_dir = "/workspace/maniskill_project/rgbd_data/"
os.makedirs(output_dir, exist_ok=True)

print("=" * 60)
print("REPLAYING TRAJECTORY WITH RGBD")
print("(With proper state initialization!)")
print("=" * 60)

In [ ]:
# Load trajectory metadata to get control mode
with open(json_path, 'r') as f:
    traj_meta = json.load(f)

control_mode = traj_meta.get('env_kwargs', {}).get('control_mode', 'pd_joint_delta_pos')
print(f"Control mode from trajectory: {control_mode}")

In [ ]:
# Create environment with MATCHING control mode
print("\n1. Creating environment with RGBD observations...")

env = gym.make(
    "PickCube-v1",
    obs_mode="rgbd",
    control_mode=control_mode,  # MUST match the recorded trajectory!
    render_mode="rgb_array",
    num_envs=1
)

print(f"   ✓ Environment created")
print(f"   Control mode: {control_mode}")
print(f"   Action space: {env.action_space}")

In [ ]:
def replay_trajectory(env, traj_path, traj_idx=0, save_video=True, output_dir="./"):
    """
    Replay a single trajectory with proper state initialization.
    
    The KEY is using env.set_state() to restore the exact initial state!
    """
    
    with h5py.File(traj_path, 'r') as f:
        traj_name = f'traj_{traj_idx}'
        if traj_name not in f:
            print(f"Trajectory {traj_name} not found!")
            return None
        
        traj = f[traj_name]
        
        # Load actions
        actions = traj['actions'][:]
        
        # Load initial state (this is the KEY!)
        # env_states has shape (T+1, state_dim) - first one is initial state
        initial_state = {}
        if 'env_states' in traj:
            env_states = traj['env_states']
            # Get the FIRST state (index 0)
            for group_name in env_states.keys():
                initial_state[group_name] = {}
                group = env_states[group_name]
                if isinstance(group, h5py.Group):
                    for obj_name in group.keys():
                        initial_state[group_name][obj_name] = group[obj_name][0]  # First timestep
    
    print(f"\n   Replaying {traj_name}: {len(actions)} actions")
    
    # Reset environment first
    obs, _ = env.reset(seed=traj_idx)
    
    # Set the environment to the EXACT initial state from the trajectory
    # This is crucial! Without this, the robot starts in wrong position
    try:
        env.unwrapped.set_state_dict(initial_state)
        print(f"   ✓ Set initial state from trajectory")
    except Exception as e:
        print(f"   ⚠ Could not set state: {e}")
        print(f"   Using env.reset() state instead")
    
    # Get observation after setting state
    obs = env.unwrapped.get_obs()
    
    # Storage
    all_rgb = []
    all_depth = []
    frames = []
    
    # Get camera name
    camera_name = list(obs['sensor_data'].keys())[0]
    
    # Helper to extract numpy from torch tensors
    def to_numpy(x):
        if hasattr(x, 'cpu'):
            x = x.cpu().numpy()
        if len(x.shape) == 4:  # Batch dim
            x = x[0]
        return x
    
    # Capture initial frame
    rgb = to_numpy(obs['sensor_data'][camera_name]['rgb'])
    depth = to_numpy(obs['sensor_data'][camera_name]['depth'])
    all_rgb.append(rgb)
    all_depth.append(depth)
    
    frame = env.render()
    frames.append(to_numpy(frame))
    
    # Replay all actions
    for action in tqdm(actions, desc=f"   Replaying", leave=False):
        # Prepare action
        action_tensor = np.zeros(env.action_space.shape, dtype=np.float32)
        action_tensor[:len(action)] = action
        
        # Step
        obs, reward, terminated, truncated, info = env.step(action_tensor)
        
        # Capture RGBD
        rgb = to_numpy(obs['sensor_data'][camera_name]['rgb'])
        depth = to_numpy(obs['sensor_data'][camera_name]['depth'])
        all_rgb.append(rgb)
        all_depth.append(depth)
        
        # Render
        frame = env.render()
        frames.append(to_numpy(frame))
        
        if terminated or truncated:
            break
    
    # Save video
    if save_video:
        video_path = os.path.join(output_dir, f"{traj_name}_replay.mp4")
        imageio.mimsave(video_path, frames, fps=30)
        print(f"   ✓ Video saved: {video_path}")
    
    return {
        'rgb': np.array(all_rgb),
        'depth': np.array(all_depth),
        'frames': frames,
        'traj_idx': traj_idx
    }

In [ ]:
# Replay trajectory 0
print("\n2. Replaying trajectory 0...")
result = replay_trajectory(env, traj_path, traj_idx=0, save_video=True, output_dir=output_dir)

if result:
    print(f"\n   ✓ Captured {len(result['rgb'])} RGBD observations!")
    print(f"   RGB shape: {result['rgb'].shape}")
    print(f"   Depth shape: {result['depth'].shape}")

---
## 5. Visualize RGB and Depth Images

In [ ]:
import matplotlib.pyplot as plt

if result:
    all_rgb = result['rgb']
    all_depth = result['depth']
    
    # Pick frames to visualize
    frame_indices = [0, len(all_rgb)//4, len(all_rgb)//2, 3*len(all_rgb)//4, len(all_rgb)-1]
    
    fig, axes = plt.subplots(2, len(frame_indices), figsize=(20, 8))
    
    for i, idx in enumerate(frame_indices):
        # RGB
        axes[0, i].imshow(all_rgb[idx])
        axes[0, i].set_title(f'RGB - Step {idx}')
        axes[0, i].axis('off')
        
        # Depth
        depth_img = all_depth[idx]
        if len(depth_img.shape) == 3:
            depth_img = depth_img[:, :, 0]  # Remove channel dim
        axes[1, i].imshow(depth_img, cmap='viridis')
        axes[1, i].set_title(f'Depth - Step {idx}')
        axes[1, i].axis('off')
    
    plt.suptitle('Trajectory Replay - RGB and Depth Observations', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'rgbd_visualization.png'), dpi=150)
    plt.show()
    
    print(f"\n✓ Visualization saved!")

---
## 6. Watch the Video

In [ ]:
from IPython.display import Video, display

video_path = "/workspace/maniskill_project/rgbd_data/traj_0_replay.mp4"

print("Replayed Trajectory Video:")
print("(Robot picking up the cube using saved expert actions)\n")

display(Video(video_path, embed=True, width=640))

---
## 7. Replay 50 Trajectories and View in Grid

Now let's replay multiple trajectories and see them all!

In [ ]:
# Replay 50 trajectories
NUM_TRAJECTORIES = 50

print("=" * 60)
print(f"REPLAYING {NUM_TRAJECTORIES} TRAJECTORIES")
print("=" * 60)

video_paths = []

for i in tqdm(range(NUM_TRAJECTORIES), desc="Replaying trajectories"):
    result = replay_trajectory(
        env, 
        traj_path, 
        traj_idx=i, 
        save_video=True, 
        output_dir=output_dir
    )
    if result:
        video_paths.append(os.path.join(output_dir, f"traj_{i}_replay.mp4"))

print(f"\n✓ Saved {len(video_paths)} videos!")

In [ ]:
# Close environment
env.close()
print("Environment closed.")

In [ ]:
# Create a grid of first frames from each video
import matplotlib.pyplot as plt
import imageio

print("Creating grid of trajectory first frames...")

# Load first frame from each video
first_frames = []
for vp in video_paths[:50]:  # Limit to 50
    if os.path.exists(vp):
        reader = imageio.get_reader(vp)
        first_frames.append(reader.get_data(0))
        reader.close()

# Create grid
n_cols = 10
n_rows = (len(first_frames) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 2*n_rows))
axes = axes.flatten()

for i, frame in enumerate(first_frames):
    axes[i].imshow(frame)
    axes[i].set_title(f'Traj {i}', fontsize=8)
    axes[i].axis('off')

# Hide unused subplots
for i in range(len(first_frames), len(axes)):
    axes[i].axis('off')

plt.suptitle(f'First Frame of {len(first_frames)} Trajectories', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'all_trajectories_grid.png'), dpi=150)
plt.show()

print(f"✓ Grid saved to {output_dir}all_trajectories_grid.png")

In [ ]:
# Watch any specific trajectory
from IPython.display import Video, display
import ipywidgets as widgets

def show_trajectory(traj_num):
    video_path = f"/workspace/maniskill_project/rgbd_data/traj_{traj_num}_replay.mp4"
    if os.path.exists(video_path):
        display(Video(video_path, embed=True, width=480))
    else:
        print(f"Video not found: {video_path}")

print("Select a trajectory to watch:")
print("(Change the number and run the cell again)")
print()

# Show trajectory 0 by default
show_trajectory(0)

In [ ]:
# Show multiple videos in a row
from IPython.display import HTML, display

# Create HTML to display 5 videos side by side
html_content = '<div style="display: flex; flex-wrap: wrap; gap: 10px;">'

for i in range(min(5, len(video_paths))):
    vp = video_paths[i]
    html_content += f'''
    <div style="text-align: center;">
        <video width="200" controls>
            <source src="{vp}" type="video/mp4">
        </video>
        <p>Trajectory {i}</p>
    </div>
    '''

html_content += '</div>'

print("First 5 trajectories:")
display(HTML(html_content))

In [ ]:
# List all generated files
print("=" * 60)
print("ALL GENERATED FILES")
print("=" * 60)
print(f"\nDirectory: {output_dir}")
print()

files = sorted(os.listdir(output_dir))
videos = [f for f in files if f.endswith('.mp4')]
images = [f for f in files if f.endswith('.png')]

print(f"Videos ({len(videos)}):")
for v in videos[:10]:
    print(f"  - {v}")
if len(videos) > 10:
    print(f"  ... and {len(videos)-10} more")

print(f"\nImages ({len(images)}):")
for img in images:
    print(f"  - {img}")